# Solving Sudoku with exact cover

Sudoku solved by exact cover -algorithm implemented by QUBO in quantum annealer.

Sudoku can be converted to exact cover problem by following procedure:

- Elements of set $U$ in exact cover problem represent here sudoku rule restricition. For example: cell $(y,x)$ of the sudoku is filled, block $y/3,x/3$ has number $n$ somewhere in block, row $y$ has number $n$, and column $x$ has number $n$. So there are altogether $4*9*9=324$ elements in $U$.
- Every subset $V_i \in V$ is an option to fill a cell in sudoku: A number $n$ is placed to a cell $(y,x)$. All these subsets have four elements: cell $(y,x)$ is filled, block $y/3,x/3$ has number $n$, row $y$ has number $n$, and column $x$ has number $n$.
- To reduce the needed qubits some “evidently impossible” choices of $n$ to cell $(y,x)$ are not included in subsets $V$. For example number $n$ is already in that row.

After this problem is solved by the exact cover algorithm with set $V$.

In [3]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
from minorminer.busclique import find_clique_embedding

## Some helper functions

In [59]:
stoi = {'.':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'A':10, 'B':11, 'C':12, 'D':13, 'E':14, 'F':15, 'G':16} 
itos = {0:'.', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'A', 11:'B', 12:'C', 13:'D', 14:'E', 15:'F', 16:'G'} 

def count_zeros(sudoku):
    count=0;
    for y in range(size2):
        for x in range(size2):
            if sudoku[y][x]==0:
                count += 1
    return count

def print_sudoku(sudoku):
    for y in range(size2):
        if y!=0 and y%ssize==0:
            if ssize==4: print('----+----+----+----')      
            if ssize==3: print('---+---+---')      
            if ssize==2: print('--+--')      
        for x in range(size2):
            if x!=0 and x%ssize==0:
                print('|',end='')
            print(itos[sudoku[y][x]], end='')
        print('')
        
def merge_sudoku(sudoku1, sudoku2):
    sudoku_res = np.zeros((ssize*ssize,ssize*ssize))
    for y in range(size2):
        for x in range(size2):
            if sudoku1[y][x]==0:
                sudoku_res[y][x] = sudoku2[y][x]
            else:
                sudoku_res[y][x] = sudoku1[y][x]
                if sudoku2[y][x]>0:
                    print('! Sudokus have overlapping cell: {},{}'.format(y+1,x+1))
    return sudoku_res

def check_sudoku(sudoku):
    f = 0
    for i in range(size2):
        g = []
        for j in range(size2):
            g.append(sudoku[i][j])
        for n in range(1,size2+1):
            if not n in g:
                print('Number {} missing in row {}.'.format(itos[n],i+1))
                f += 1
    for i in range(size2):
        g = []
        for j in range(size2):
            g.append(sudoku[j][i])
        for n in range(1,size2+1):
            if not n in g:
                print('Number {} missing in column {}.'.format(itos[n],i+1))
                f += 1
    for i1 in range(ssize):
        for i2 in range(ssize):
            g = []
            for j1 in range(ssize):
                for j2 in range(ssize):
                    g.append(sudoku[i1*ssize+j1][i2*ssize+j2])
            for n in range(1,size2+1):
                if not n in g:
                    print('Number {} missing in block {},{}.'.format(itos[n],i1+1,i2+1))
                    f += 1
    if f==0:
         print('sudoku OK')
    else:
        print('number of problems:',f)

def check_sudoku_rules(sudoku):
    ga = 0;
    for n in range(1, size2+1):
        for i in range(size2):
            g = 0
            for j in range(size2):
                if sudoku[i][j]==n:
                    g += 1
            if g>1:
                print('Number {} more then once in row {}.'.format(itos[n],i+1))
                ga += 1
        for i in range(size2):
            g = 0
            for j in range(size2):
                if sudoku[j][i]==n:
                    g += 1
            if g>1:
                print('Number {} more then once in column {}.'.format(itos[n],i+1))
                ga += 1
        for i1 in range(ssize):
            for i2 in range(ssize):
                g = 0
                for j1 in range(ssize):
                    for j2 in range(ssize):
                        if sudoku[i1*ssize+j1][i2*ssize+j2]==n:
                            g += 1
                if g>1:
                    print('Number {} more than once in block {},{}.'.format(itos[n],i1+1,i2+1))
                    ga += 1
    if ga==0:
         print('Rules obeyed.')
    else:
        print('number of problems:',ga)

## Load sudoku

sudoku file structure: first line is number of sudokus. First line of every sudoku is the size of sudoku.

3_level_48.ss: These should be rather easy sudokus, 48 cells not filled.

In [62]:
#file = '3_level_48.ss'
#file = '2_level_varia.ss'
#file = '3_worldsHardest_60.ss'
file = '4_level_163.ss'
f = open('testdata/'+file,'r')

scount = int(f.readline())
ssize = int(f.readline())

sudoku=np.zeros((scount,ssize*ssize,ssize*ssize))

for i in range(scount):
    for j in range(ssize):        
        for k in range(ssize):
            line = f.readline()
            for l in range(ssize*ssize):
                c = line[l+int(l/ssize)]
                sudoku[i][j*ssize+k][l] = stoi[c]
        line = f.readline()

## Create set and subset from sudoku

Sudoku can be converted to exact cover problem:
- set elements are all option in all cells. So in empty sudoku there is 9x9x9 elements. To get little bit better efficiency, here only empy cells are counted in.
- subset block: 1) every cell can have only one number, 2) every 3x3 block can have only one number each, 3) every row can have only one number each, 4) every column can have only one number each

Every four subset block has 81 subsets like this: 1) there is 81 cells in sudoku, 2) sudoku has 9 blocks, for wich one subset for each number, 3) 9 rows, for wich one subset for each number, 4) 9 columns, for wich one subset for each number.

Some preprocessing is made, known numbers are one large subset.

In [63]:
sind = 0             # which sudoku from the list
size2 = ssize*ssize

U = [i for i in range(size2*size2*4)]  # 4 blocks of subsets
V = []                                  # Set element names are row*81 + col*9 + "1-9"
cell_id = []

t1 = time.time()

# “evidently impossible” choices
c = []
for y in range(size2):
    for x in range(size2):
        if sudoku[sind][y][x]!=0:
            n = sudoku[sind][y][x]-1
            c.append(int(y*size2+x))
            c.append(int(size2*size2 + int(y/ssize)*size2*ssize + int(x/ssize)*size2 + n))
            c.append(int(2*size2*size2 + y*size2 + n))
            c.append(int(3*size2*size2 + x*size2 + n))
c.sort()
            
y=0
x=0
while y<size2:
    if sudoku[sind][y][x]==0:
        for j in range(size2):
            # cell, block, row, column, restriction
            bl = size2*size2 + int(y/ssize)*size2*ssize + int(x/ssize)*size2 + j
            row = 2*size2*size2 + y*size2 + j
            col = 3*size2*size2 + x*size2 + j
            if not (bl in c or row in c or col in c):
                V.append([y*size2+x, bl, row, col])
                cell_id.append((y,x,j+1))
    x += 1
    if x==size2:
        x = 0
        y += 1

z = count_zeros(sudoku[sind])
t2 = time.time()
print('Time used for constructing Q (ms): {:.1f}'.format((t2-t1)*1000))
print('Number of blanks:',z)            
print('Number of elements:',len(U))
print('Number of subsets:',len(V))
print('Number of subsets if no zipping:',size2*size2*size2)

Time used for constructing Q (ms): 17.0
Number of blanks: 163
Number of elements: 1024
Number of subsets: 854
Number of subsets if no zipping: 4096


## Create QUBO

Constraints
- total number of elements in subset should be $|U|$
- each element only in one subset

In [64]:
Q = np.zeros((len(V),len(V)))
t1 = time.time()

# Total elements constraint
for i in range(len(V)):
    Q[i][i] = -1

# each element only in one subset
for a in U:
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            if a in V[i] and a in V[j]:
                Q[i][j] = 3
                
t2 = time.time()
print('Time used for constructing QUBO (ms): {:.1f}'.format((t2-t1)*1000))

Time used for constructing QUBO (ms): 41796.6


## Creat BQM from QUBO

In [65]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
print('Number of logical qubits needed:',Q.shape[0])
print('Number of logical couplers needed:', len(bqm.quadratic))

Number of logical qubits needed: 854
Number of logical couplers needed: 7466


## Local heuristic classical solver

Local deterministic solver can not be used because: "Maximum allowed dimension exceeded"

In [66]:
num_reads = 1000
t1 = time.time()
sampleset = SimulatedAnnealingSampler().sample(bqm, num_reads=num_reads).aggregate()
t2 = time.time()
print('Time used by solver (ms): {:.1f}'.format((t2-t1)*100))
print('Lowest energy reached:',int(sampleset.first.energy))
print('Lowest energy should be:',-count_zeros(sudoku[sind]))   
print('Lowest energy occurences: {} %'.format(int(sampleset.first.num_occurrences/num_reads*100)))

Time used by solver (ms): 1828.3
Lowest energy reached: -153
Lowest energy should be: -163
Lowest energy occurences: 0 %


### Results

In [67]:
sudoku_res = np.zeros((ssize*ssize,ssize*ssize))
r = sampleset.first.sample
for k,v in r.items():
    if v==1 and k<len(cell_id):
        y,x,n = cell_id[k]
        sudoku_res[y][x] = n

In [68]:
print_sudoku(sudoku[sind])

9..1|4..7|...B|.G2.
...D|9C2.|4.E.|A.58
6..5|.E..|..92|...7
G..B|..6.|.A..|..3D
----+----+----+----
....|F3.G|8..6|7.C.
..7C|...E|AF4.|....
.2AG|....|.C.1|.6E.
....|B...|...E|....
----+----+----+----
....|.A.9|.1.4|....
..B.|8...|D.3.|F.GA
..F.|6..5|..7.|.1..
....|.2.F|..58|C4..
----+----+----+----
F6..|..B.|..C7|G...
B.1.|..4C|.E.5|9D.6
..59|.7..|....|....
.E.4|....|....|.7..


In [69]:
print_sudoku(sudoku_res)

.AE.|.DF.|538.|6..C
73..|...B|.6.G|.F..
.48.|A.G3|FD..|1B..
.F2.|5..8|7.1C|4E..
----+----+----+----
DB4E|..A.|.52.|.9.1
58..|D61.|...3|.2BG
3...|7854|9.B.|D..F
1.6F|.9C2|G7D.|8345
----+----+----+----
85G7|C.E.|6.F.|2.DB
EC.6|.471|.2.9|.5..
49.2|.BD.|CG.A|E.83
A1D3|G...|EB..|..69
----+----+----+----
..3A|15.D|24..|.8.E
.7.8|2G..|3.A.|..F.
2D..|E..A|B86F|3C14
C...|3F86|19GD|B.A2


In [70]:
merge = merge_sudoku(sudoku[sind],sudoku_res)
print_sudoku(merge)

9AE1|4DF7|538B|6G2C
73.D|9C2B|46EG|AF58
6485|AEG3|FD92|1B.7
GF2B|5.68|7A1C|4E3D
----+----+----+----
DB4E|F3AG|8526|79C1
587C|D61E|AF43|.2BG
32AG|7854|9CB1|D6EF
1.6F|B9C2|G7DE|8345
----+----+----+----
85G7|CAE9|61F4|2.DB
ECB6|8471|D239|F5GA
49F2|6BD5|CG7A|E183
A1D3|G2.F|EB58|C469
----+----+----+----
F63A|15BD|24C7|G8.E
B718|2G4C|3EA5|9DF6
2D59|E7.A|B86F|3C14
CE.4|3F86|19GD|B7A2


In [71]:
check_sudoku(merge)

Number 1 missing in row 2.
Number C missing in row 3.
Number 9 missing in row 4.
Number 9 missing in row 6.
Number A missing in row 8.
Number 3 missing in row 9.
Number 7 missing in row 12.
Number 9 missing in row 13.
Number G missing in row 15.
Number 5 missing in row 16.
Number G missing in column 2.
Number 9 missing in column 3.
Number C missing in column 3.
Number 1 missing in column 6.
Number 3 missing in column 7.
Number 9 missing in column 7.
Number 5 missing in column 13.
Number A missing in column 14.
Number 7 missing in column 15.
Number 9 missing in column 15.
Number C missing in block 1,1.
Number 1 missing in block 1,2.
Number 9 missing in block 1,4.
Number 9 missing in block 2,1.
Number A missing in block 2,4.
Number 3 missing in block 3,2.
Number 7 missing in block 3,4.
Number G missing in block 4,1.
Number 9 missing in block 4,2.
Number 5 missing in block 4,4.
number of problems: 30


In [72]:
check_sudoku_rules(merge)

Rules obeyed.


# Quantum solver

In [48]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [78]:
num_reads = 1000

embedding = find_clique_embedding(bqm.variables, machine.to_networkx_graph())  
num_qubits_needed = sum(len(chain) for chain in embedding.values())
print('Estimate actual qubits needed:',num_qubits_needed)

anneal_schedule = [[0.0, 0.0], [40.0, 0.4], [1040.0, 0.4], [1042, 1.0]]
estimated_runtime = machine.solver.estimate_qpu_access_time(num_qubits_needed, num_reads=num_reads, anneal_schedule=anneal_schedule)    
print("Estimate QPU time needed (ms) : {:.0f}".format(estimated_runtime/1000, machine.solver.name)) 

Estimate actual qubits needed: 1952
Estimate QPU time needed (ms) : 1313


In [50]:
sampleset2 = EmbeddingComposite(machine).sample(bqm, num_reads=num_reads)

In [77]:
qtime = sampleset2.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset2.info['embedding_context']['embedding'].values())
print('Lowest energy should be:',-count_zeros(sudoku[sind]))  
print('Lowest energy reached:',int(sampleset2.first.energy))
print('Lowest energy occurences: {:.1f} %'.format(sampleset2.first.num_occurrences/num_reads*100))
print('QPU time used (ms): {:.1f}'.format(qtime))
print('Physical qubits used: {}'.format(qubits))

Lowest energy should be: -48
Lowest energy reached: -37
Lowest energy occurences: 0.1 %
QPU time used (ms): 160.7
Physical qubits used: 595


### Results

In [52]:
sudoku_res2 = np.zeros((ssize*ssize,ssize*ssize))
r = sampleset2.first.sample
for k,v in r.items():
    if v==1 and k<len(cell_id):
        y,x,n = cell_id[k]
        sudoku_res2[y][x] = n

In [53]:
print_sudoku(sudoku[sind])

..9|..3|.1.
..1|.27|.8.
...|15.|963
---+---+---
5.8|..1|.49
94.|..6|.3.
...|..9|...
---+---+---
726|3.8|1.4
.34|.6.|..2
...|...|...


In [54]:
print_sudoku(sudoku_res2)

4..|68.|5..
65.|9..|4..
287|..4|...
---+---+---
.6.|.7.|2..
...|2..|..1
312|84.|6.7
---+---+---
...|.9.|.5.
1..|7.5|.9.
895|412|376


In [56]:
merge2 = merge_sudoku(sudoku[sind],sudoku_res2)
print_sudoku(merge2)

4.9|683|51.
651|927|48.
287|154|963
---+---+---
568|.71|249
94.|2.6|.31
312|849|6.7
---+---+---
726|398|154
134|765|.92
895|412|376


In [57]:
check_sudoku_rules(merge2)

Rules obeyed.


In [66]:
print('Cells missing:',count_zeros(merge2))

Cells missing: 9


In [68]:
dwave.inspector.show(sampleset2)

Serving Inspector on http://127.0.0.1:18001/?problemId=0ad0f018-5571-4898-b95c-8ac3367f548d

'http://127.0.0.1:18001/?problemId=0ad0f018-5571-4898-b95c-8ac3367f548d'

In [67]:
print(sampleset2.truncate(30))

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 140 energy num_oc. ...
0   0  1  0  0  0  0  0  0  0  1  0  0  1  0  0  1 ...   0  -37.0       1 ...
1   1  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0 ...   0  -36.0       1 ...
2   0  0  0  0  0  0  1  0  0  1  0  0  1  0  1  0 ...   0  -36.0       1 ...
3   0  0  1  0  1  0  0  0  0  0  1  1  0  1  0  0 ...   0  -35.0       1 ...
4   1  0  0  0  0  0  0  0  0  1  0  0  0  0  1  0 ...   0  -35.0       1 ...
5   1  0  0  0  0  0  0  0  0  1  0  0  1  0  1  0 ...   1  -35.0       1 ...
6   0  0  0  0  1  0  0  0  0  1  0  0  0  0  1  0 ...   0  -35.0       1 ...
7   1  0  0  0  0  1  0  0  0  0  0  0  1  0  1  0 ...   0  -34.0       1 ...
8   0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  1 ...   0  -34.0       1 ...
9   0  0  1  0  0  0  0  0  0  0  1  0  0  1  0  0 ...   1  -34.0       1 ...
10  1  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0 ...   1  -34.0       1 ...
11  0  1  0  0  1  0  0  0  0  1  0  0  1  1  0  0 ...   1  -34.

## Hybrid solver

In [79]:
sampleset2 = LeapHybridSampler().sample(bqm)

In [80]:
print('Lowest energy should be:',-count_zeros(sudoku[sind]))  
print('Lowest energy reached:',int(sampleset2.first.energy))

Lowest energy should be: -163
Lowest energy reached: -161


In [81]:
hyb_time = sampleset2.info['qpu_access_time'] / 1000
run_time = sampleset2.info['run_time'] / 1000
print('QPU time used (ms): {:.1f}'.format(hyb_time))
print('Total time used (ms): {:.1f}\n'.format(run_time))
print(sampleset2) 

QPU time used (ms): 85.4
Total time used (ms): 2987.1

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 ... 853 energy num_oc.
0  0  0  1  0  0  0  0  0  1  0  0  0  1  0  0  0  0  1 ...   0 -161.0       1
['BINARY', 1 rows, 1 samples, 854 variables]


In [82]:
sudoku_res2 = np.zeros((ssize*ssize,ssize*ssize))
r = sampleset2.first.sample
for k,v in r.items():
    if v==1 and k<len(cell_id):
        y,x,n = cell_id[k]
        sudoku_res2[y][x] = n

In [83]:
merge2 = merge_sudoku(sudoku[sind],sudoku_res2)
print_sudoku(merge2)

9AE1|4FD7|538B|6G2C
7F.D|9C23|46EG|AB58
6835|GEAB|CD92|1F47
G4CB|.568|7A1F|E93D
----+----+----+----
DB4E|F39G|8526|7AC1
597C|261E|AF43|D8BG
82AG|7D54|9CB1|36EF
136F|B8CA|G7DE|4592
----+----+----+----
2DG8|CAE9|F164|B375
E5B6|8471|D93C|F2GA
4CF3|6BG5|E27A|81D9
A197|D23F|BG58|C46E
----+----+----+----
F68A|59BD|24C7|GE13
B712|AG4C|3EF5|9D86
3G59|E786|1BAD|2CF4
CED4|31F2|68G9|57AB


In [84]:
check_sudoku_rules(merge2)
print('Cells missing:',count_zeros(merge2))

Rules obeyed.
Cells missing: 2
